# 2. Podcast Listening Time - Modeling

We have done the Exploratory Data Analysis in the [Last Notebook](1_Podcast_EDA.ipynb), now it's time to actually start to model the Listening Time.

# Base Estimators

In [1]:
import warnings
warnings.filterwarnings(
    "ignore", message=r".*A worker stopped while some jobs were given to the executor.*",
    category=UserWarning, module=r"joblib[.]externals[.]loky[.]process_executor"
)
warnings.filterwarnings(
    "ignore",
    message=r".*Using `tqdm.autonotebook.tqdm` in notebook mode.*",
    module=r"tqdm_joblib"
)

import os
import pandas as pd
import warnings
import math
import joblib
from tqdm.notebook import tqdm
from tqdm_joblib import tqdm_joblib
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from sklearn import set_config
from podcast_functions import *
from podcast_models import *
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score,root_mean_squared_error


result_folder = "Results"
os.makedirs(result_folder, exist_ok=True)
set_config(transform_output="pandas")   

# Load data
df_train = pd.read_csv(r"Data/train.csv")
df_test = pd.read_csv(r"Data/test.csv") 

target = "Listening_Time_minutes"
X_cat = ["Podcast_Name","Publication_Day", "Publication_Time", "Episode_Title","Episode_Sentiment"]
X_num = ["Episode_Length_minutes","Guest_Popularity_percentage","Number_of_Ads","Host_Popularity_percentage"]

# Set categorical columns to 'category' dtype
for col in X_cat:
    df_train[col] = df_train[col].astype('category')
    df_test[col]  = df_test[col].astype('category')
    
X = df_train[X_cat + X_num]
y = df_train[target]

The goal is to minimize the RMSE since it's the metric used in the competition. We already explained in the EDA the steps in the preprocessors aiming at cleaning the Data (removing outliers, capping data...).

The other steps in the preprocessor vary depending on the algorithms as some of them need imputing missing values, encoding categorical variables or scaling numeric variables.

In [5]:
pre = {
    "base": make_preprocessor(X_num, X_cat),
    "imp": make_preprocessor(X_num, X_cat, impute=True),
    "imp_onehot": make_preprocessor(X_num, X_cat, onehot=True, impute=True, sparse_output=True, scaler=True),
    "imp_dense": make_preprocessor(X_num, X_cat, onehot=True, impute=True, sparse_output=False),
    "dense": make_preprocessor(X_num, X_cat, onehot=True, sparse_output=False),
    "imp_dense_sc": make_preprocessor(X_num, X_cat, onehot=True, impute=True, sparse_output=False, scaler=True),
    "imp_ordinal": make_preprocessor(X_num, X_cat, impute=True, ordinal=True),
}

We are going to use a diverse set of estimators to capture the maximum of variance possible. Each pipeline is tune with cross-validation and model-appropriate search strategies:
- Fast and Strong baselines Gradient Boosting Models: **LightGBM, XGBoost, CatBoost and HistGradientBoosting**
- Bagging / Randomized trees: **Random Forest and ExtraTrees**
- Purely Additive Boosting : **Explainable Boosting Machine**
- Linear and Distance/Margin models : **ElasticNet, k NN and SVR (RBF)**
- Neural-based tabular models : **Shallow MLP and TabNet**

In [3]:
# --- Unified config list ---
configs = [
    ("lgbm",run_lightGBM,      {"preprocessor": pre["base"]}),
    ("xgb", run_XGBoost,       {"preprocessor": pre["base"]}),
    ("cat", run_CatBoost,      {"preprocessor": pre["base"], "X_cat": X_cat}),
    ("hgb", run_HGB,           {"preprocessor": pre["imp_dense"]}),
    ("rf",  run_randomForest,  {"preprocessor": pre["dense"]}),
    ("et",  run_extraTrees,    {"preprocessor": pre["imp_dense"], "n_candidates": 20, "n_jobs": 8}),
    ("ebm", run_EBM,           {"preprocessor": pre["imp"]}),
    ("enet",run_ElasticNet,    {"preprocessor": pre["imp_onehot"], "n_jobs": 16}),
    ("knn", run_KNN,           {"preprocessor": pre["imp_dense_sc"], "n_jobs": 16}),
    ("svr", run_SVR,           {"preprocessor": pre["imp_dense_sc"],  "n_candidates": 25, "n_jobs": 8}),
    ("nn",  run_NeuralNetwork, {"preprocessor": pre["imp_dense"], "n_jobs": 8}),
    ("tab", run_tabnet,        {"X_num": X_num, "X_cat": X_cat, "preprocessor": pre["imp_ordinal"]}),
]

# --- Run models or load them from disk ---
model_folder = os.path.join(result_folder, "models")
os.makedirs(model_folder, exist_ok=True)

overwrite_model = False
models = {}

for key, func, kw in configs:
    
    model_path = os.path.join(model_folder, f"models_{key}.joblib")
    if overwrite_model or not os.path.exists(model_path):
        print(f"Running {key}")
        search = func(X=X, y=y, **kw)
        joblib.dump(search, model_path, compress=3)
        models[key] = search
    else:
        print(f"Loading {key}")
        models[key] = joblib.load(model_path)


# --- Write Predictions ---
pred_folder = os.path.join(result_folder, "predictions")
os.makedirs(pred_folder, exist_ok=True)

for model_name, search in models.items():
    warnings.filterwarnings("ignore", category=UserWarning)
    write_predictions(
        df=df_test,
        model=search.best_estimator_,
        features=X_cat + X_num,
        target=target,
        path= os.path.join(pred_folder, f"predictions_{model_name}.csv")
    )

Loading lgbm
Loading xgb
Loading cat
Loading hgb
Loading rf
Loading et
Loading ebm
Loading enet
Loading knn
Loading svr
Loading nn
Loading tab


We have a good idea about the ranking of our different models, even though we have to be careful as these are the results of the Cross Validation, we will get the *real* results after we submit our predictions to Kaggle.

In [4]:
# --- Write Predictions ---
cv_folder = os.path.join(result_folder, "cross_validation")
os.makedirs(cv_folder, exist_ok=True)

overwrite_cv = False

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
rmse_list = []
r2_list = []
oof_list = []

for model_name, search in models.items():
    np_path = os.path.join(cv_folder, f"cv_{model_name}.npy")
    best = search.best_estimator_
    if overwrite_cv or not os.path.exists(np_path):
        oof_pred = cross_val_predict(best, X, y, cv=cv, n_jobs=3)
        np.save(np_path, oof_pred)
    else:
        oof_pred = np.load(np_path)
        
    oof_r2 = r2_score(y, oof_pred)
    oof_rmse = root_mean_squared_error(y, oof_pred)
    
    oof_list.append(oof_pred)
    r2_list.append(oof_r2)
    rmse_list.append(oof_rmse)
    print(f"{model_name.upper()} CV RMSE: {oof_rmse:.6f} / R2 : {oof_r2:.6f}")
    
df_rmse = pd.DataFrame({
    'model': list(models.keys()),
    'rmse': rmse_list,
    'r2': r2_list 
})

matrix_oof = np.vstack(oof_list)
matrix_oof_T = matrix_oof.T
df_oof = pd.DataFrame(matrix_oof_T)
df_oof.columns = list(models.keys())

LGBM CV RMSE: 12.689524 / R2 : 0.781362
XGB CV RMSE: 12.738678 / R2 : 0.779665
CAT CV RMSE: 12.980260 / R2 : 0.771229
HGB CV RMSE: 13.010592 / R2 : 0.770158
RF CV RMSE: 12.637614 / R2 : 0.783147
ET CV RMSE: 12.692488 / R2 : 0.781260
EBM CV RMSE: 13.074538 / R2 : 0.767893
ENET CV RMSE: 13.346651 / R2 : 0.758131
KNN CV RMSE: 12.995897 / R2 : 0.770677
SVR CV RMSE: 13.266125 / R2 : 0.761041
NN CV RMSE: 13.273978 / R2 : 0.760758
TAB CV RMSE: 13.255676 / R2 : 0.761417


# Stacking

After identifying strong individual learners, we build a **StackingRegressor** where the final estimator is LightGBM as it allows to
- Weight the different mistakes of the single models
- Blend high-bias and high-variance learners to reduce RMSE without over-smoothing

In [5]:
overwrite_stack = False
stack_path = os.path.join(model_folder, "models_stack.joblib")

if overwrite_stack or not os.path.exists(stack_path):
    stack = StackingRegressor(
        estimators=[(model_name, search.best_estimator_) for model_name, search in models.items()],
        final_estimator=LGBMRegressor(
            n_estimators=2000,
            learning_rate=0.02,
            max_depth=3,        
            subsample=0.7,
            colsample_bytree=0.7,
            random_state=SEED,
            verbosity=-1
        ),
        cv=cv,
        n_jobs=1,
        verbose=2
    )

    with tqdm_joblib(tqdm(total=2 * len(models), desc="GridSearch Stacking")):
        stack.fit(X, y)
    joblib.dump(stack,stack_path , compress=3)
else:
    stack = joblib.load(stack_path)
    
write_predictions(
    df=df_test,model=stack,features=X_cat + X_num,
    target=target,path= os.path.join(pred_folder, "predictions_stack.csv")
)

#### Stack Cross Validation

In [6]:
overwrite_stack_cv = False
stack_cv_path = os.path.join(cv_folder, "cv_stack.npy")

if overwrite_stack_cv or not os.path.exists(stack_cv_path):
    with tqdm_joblib(tqdm(total=cv.get_n_splits(), desc="GridSearch Cross Val Predict")):
        oof_stack = cross_val_predict(stack, X, y,cv=cv, n_jobs=2,method='predict',verbose=10)
        np.save(stack_cv_path, oof_stack)
else:
    oof_stack = np.load(stack_cv_path)
    
rmse_stack = root_mean_squared_error(y, oof_stack)
r2_stack   = r2_score(y, oof_stack)

In [7]:
# Update the results data frames with the stack
df_rmse.loc[len(df_rmse)] = ["stack", rmse_stack, r2_stack]
df_oof["stack"] = oof_stack
df_rmse.to_csv(os.path.join(result_folder, "cv_score.csv"), index=False)  
df_oof.to_csv(os.path.join(result_folder, "cv_oof.csv"), index=False)  

In [8]:
df_rmse

,model,rmse,r2
0,lgbm,12.689524,0.781362
1,xgb,12.738678,0.779665
2,cat,12.980260,0.771229
3,hgb,13.010592,0.770158
4,rf,12.637614,0.783147
5,et,12.692488,0.781260
6,ebm,13.074538,0.767893
7,enet,13.346651,0.758131
8,knn,12.995897,0.770677
9,svr,13.266125,0.761041


Now that our models are ready, we are going to analyze them in the [Final Analysis Notebook](3_Podcast_Final_Analysis.ipynb)